# Demonstration of EPISTEM-x Module 
This notebook contain the implementation of the source code for each module in the EPISTEM land cover mapping framework

## Library import and earth engine initialization
If you have earth engine account you could used that to authenticate and initialize the earth engine. However, if you did not have the account, service account initialization is avaliable

In [ ]:
import ee 
import epistemx

#Option 1: Manual authenticate using personal account
#Instructions for manual authentication
epistemx.print_auth_instructions()
#uncomment the below line and follow earth engine authentication process
#epistemx.authenticate_manually()

#Option 2: Autheticate using service account (json file)
service_account_path = '../auth/earth-engine-451407-4045e13555ee.json'
success = epistemx.initialize_with_service_account(service_account_path)

if success:
    print("Earth Engine initialized with service account successfully!")
else:
    print("Service account initialization failed. Try to authenticate earth engine manually")

#Check authentication status
status = epistemx.get_auth_status()
print(f"Initialized: {status['initialized']}")
print(f"Authenticated: {status['authenticated']}")
if status['project']:
    print(f"Project: {status['project']}")

2025-10-27 10:10:12,471 - epistemx.ee_config - INFO - Starting manual Earth Engine authentication...



    MANUAL EARTH ENGINE AUTHENTICATION STEPS:
    
    1. Install the Earth Engine Python API (if not already installed):
       pip install earthengine-api
    
    2. Run authentication in your terminal/command prompt:
       earthengine authenticate
    
    3. This will open a web browser. Sign in with your Google account that has Earth Engine access.
    
    4. Copy the authorization code from the browser and paste it in the terminal.
    
    5. Alternatively, you can authenticate programmatically by calling:
       from epistemx.ee_config import authenticate_manually
       authenticate_manually()
    
    6. If you don't have Earth Engine access, sign up at:
       https://earthengine.google.com/
    
    For more details, visit: https://developers.google.com/earth-engine/guides/python_install
    


2025-10-27 10:10:34,919 - epistemx.ee_config - ERROR - Manual authentication failed: Cannot authenticate: Invalid request.


False

In [ ]:
#!python -m pip install ../epistemx --quiet
import geemap
from epistemx.module_1 import Reflectance_Data, Reflectance_Stats
from epistemx.helpers import get_aoi_from_gaul

## Module 1: Acquisition of Near-Cloud-Free Satellite Imagery

### System Response 1.1: Area of Interest Definition

In [ ]:
#Set the country and province for the AOI using GAUL admin boundaries
aoi = get_aoi_from_gaul(country="Indonesia", province="Sumatera Selatan")
#Alternatively, used geemap_shp_to_ee to directly used shapefile in your local machine

### System Response 1.2: Search and Filter Imagery
The EPISTEM source code supports Landsat mission data, ranging from Landsat 1 to Landsat 9. For Landsat 1 - 3, the avaliable data is corrected radiance reflectance. The Landsat 5-9 used here is collection 2 surface reflectance (SR) analysis ready data.

The retrival logic used here is as follow:
1. Retrive multispectral bands (band 1 - 7) from landsat collection 2 SR data (if avaliable)
2. Retrive thermal band from landsat collection 2 TOA data 
3. Create temporal composite for each data 
4. Stacked the final two data into a earth engine image (ee.image)

In [ ]:
#========== FIRST RETRIVE THE MULTISPECTRAL BAND===========
#Intialize the relfectance class data function
optical_reflectance = Reflectance_Data()
#define the start and end date for imagery collection
start = '2017-01-01'
end = '2017-12-31'
#get the image collection and corresponding statistics
landsat_data, meta = optical_reflectance.get_optical_data(aoi, start, end, optical_data='L8_SR', 
                                                           cloud_cover=40, compute_detailed_stats=False)
#create mosaic between image collection, and clip based on AOI
mosaic_landsat = landsat_data.mosaic().clip(aoi)
#Alternatively you can use temporal aggregation (ee reducer) to create mode cloudless imagery
median_landsat = landsat_data.median().clip(aoi)
#visualization parameter
l8_sr_visparam = {'min': 0,'max': 0.4,'gamma': [0.95, 1.1, 1],'bands':['NIR', 'RED', 'GREEN']}
#Add the data to the map
Map = geemap.Map()
Map.addLayer(mosaic_landsat, l8_sr_visparam, 'L8 SR Mosaic')
Map.addLayer(median_landsat, l8_sr_visparam, 'L8 SR Median')
Map.addLayer(landsat_data, l8_sr_visparam, 'L8 SR Image Collection')
# set center of the map in the area of interest
Map.centerObject(aoi, 7)

2025-10-27 10:04:00,850 - Reflectance_Data - INFO - ReflectanceData initialized.
2025-10-27 10:04:00,851 - Reflectance_Data - INFO - Starting data fetch for Landsat 8 Operational Land Imager Surface Reflectance
2025-10-27 10:04:00,852 - Reflectance_Data - INFO - Date range: 2017-01-01 to 2017-12-31
2025-10-27 10:04:00,852 - Reflectance_Data - INFO - Cloud cover threshold: 40%
2025-10-27 10:04:00,853 - Reflectance_Data - INFO - detailed statistics will not be computed
2025-10-27 10:04:00,854 - Reflectance_Stats - INFO - Reflectance Stats initialized.
2025-10-27 10:04:00,855 - Reflectance_Data - INFO - Filtered collection created (use compute_detailed_stats=True for more information)


In [9]:
#retive thermal bands from TOA
thermal_bands, thermal_stats = optical_reflectance.get_thermal_bands(aoi, start, end, cloud_cover=40, compute_detailed_stats=False)
median_thermal = thermal_bands.median().clip(aoi)
thermal_vis = {
    'min': 286,
    'max': 300,
    'gammma': 0.4
}
#stacked all landsat bands
stacked_landsat = median_landsat.addBands(median_thermal)
#visualize the thermal bands and multispectral bands
Map.addLayer(median_thermal, thermal_vis, "Thermal Bands")
Map

2025-10-27 10:04:27,967 - Reflectance_Stats - INFO - Reflectance Stats initialized.
2025-10-27 10:04:27,968 - Reflectance_Data - INFO - Starting thermal data fetch for Landsat 8 Top-of-atmosphere reflectance
2025-10-27 10:04:27,970 - Reflectance_Data - INFO - Date range: 2017-01-01 to 2017-12-31
2025-10-27 10:04:27,971 - Reflectance_Data - INFO - Cloud cover threshold: 40%
2025-10-27 10:04:27,971 - Reflectance_Data - INFO - Fast mode enabled - detailed statistics will not be computed
2025-10-27 10:04:27,972 - Reflectance_Data - INFO - Filtered collection created (use compute_detailed_stats=True for detailed info)


Map(bottom=134342.0, center=[-4.075233901391136, 105.94665527343751], controls=(WidgetControl(options=['positi…

### Image retrival report (optional)

In [10]:
#intialize the statistic class
stats = Reflectance_Stats()
#get the retrival report and automatically print them
retrival_report = stats.get_collection_statistics(landsat_data, print_report=True)

2025-10-27 10:04:43,696 - Reflectance_Stats - INFO - Reflectance Stats initialized.


           Landsat Data Collection Retrival Report
Total Images Found: 53
Date Range: 2017-01-13 to 2017-12-17
Unique WRS Tiles: 10

Scene Cloud Cover Statistics:
------------------------------
Average Cloud Cover: 26.8%
Minimum Cloud Cover: 2.9%
Maximum Cloud Cover: 40.0%

WRS Path/Row Tiles:
------------------------------
Path 123/Row 062
Path 123/Row 063
Path 124/Row 061
Path 124/Row 062
Path 124/Row 063
Path 124/Row 064
Path 125/Row 061
Path 125/Row 062
Path 125/Row 063
Path 126/Row 062

Available Acqusition Date:
------------------------------
Date range: 2017-01-13 to 2017-12-17
(53 total acquisition dates)

Scene IDs (first 10):
------------------------------
• LC08_123062_20170405
• LC08_123062_20170421
• LC08_123062_20170726
• LC08_123062_20170912
• LC08_123062_20171014
• LC08_123062_20171217
• LC08_123063_20170710
• LC08_123063_20170827
• LC08_123063_20170912
• LC08_123063_20171030
... and 43 more scenes



### System Response 1.3: Imagery Download

In [ ]:
export_task = ee.batch.Export.image.toDrive(
    image=stacked_landsat,
    description='Landsat_Median_composite_2017_Sumsel',
    folder='Earth Engine',
    fileNamePrefix='Landsat_Median_composite_2017_Sumsel',
    scale=30,
    region=aoi,  # or aoi.geometry()
    maxPixels=1e13
)
export_task.start()
import time

while export_task.active():
    print('Exporting... (status: {})'.format(export_task.status()['state']))
    time.sleep(10)

print('Export complete (status: {})'.format(export_task.status()['state']))

## Module 2:  Land-cover classification Scheme